In [1]:
import nltk
import numpy as np
import random
import string

### Definir CORPUS

In [2]:
f = open("corpus.txt","r",errors='ignore')
raw = f.read()

In [3]:
# raw

### Preporcesamiento del texto con NLTK CORPUS

In [4]:
raw = raw.lower() #convertir en minuscular
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [5]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )


def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

### Evaluar Similitud Mensaje Usuario - Corpus

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords  

In [14]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

### DEFINICION DE CONINCIDENCIA MANUAL

In [43]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

### GENERACION DE RESPUESTA

In [46]:
flag = True

print("AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.")
while(flag == True):
    user_response = input()
    user_response = user_response.lower() #convertimos a minuscula

    if(user_response != 'salir'):
        if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
            flag = True
            print("AGENTE: No hay de que")
        else:
            if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                print("AGENTE: "+saludos(user_response))
            else:#si la palabra insertada no es un saludo --> Corpus
                print("AGENTE: ",end ="*")
                print(response(user_response),end="*\n")
                sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
    else:
        flag = False
        print("AGENTE: Nos vemos pronto, !Gracias por su visita¡")                
    

AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.
AGENTE: *los tapabocas los puede comprar en la farmacia.
AGENTE: Nos vemos pronto, !Gracias por su visita¡
